In [1]:
import numpy as np
import pandas as pd

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [135]:
class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None):
        
        
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 1e-10
                        num_iter: Number of Iterations to update coefficient with training data
                        early_stop: Constant control early_stop.
                        intercept: Bool, If we are going to fit a intercept, default True.
                        init_weight: Matrix (n x 1), input init_weight for testing.
                        
            
            TODO: 1. Initialize all variables needed.
        """
        
        self.model_name = 'Linear Regression'
        
        self.alpha = alpha
        self.num_iter = num_iter
        self.early_stop = early_stop
        self.intercept = intercept
        self.init_weight = init_weight  ### For testing correctness.
        
    
    def fit(self, X_train, y_train):
        """
            Save the datasets in our model, and perform gradient descent.
            
            Parameter:
                X_train: Matrix or 2-D array. Input feature matrix.
                Y_train: Matrix or 2-D array. Input target value.
                
                
                TODO: 2. If we are going to fit the intercept, add a col with all 1's to the first column. (hint: np.hstack, np.ones)
                      3. Initilaize our coef with uniform from [-1, 1] with the number of col in training set.
                      4. Call the gradient_descent function to train.
        """
        
        self.X = np.mat(X_train)
        self.y = np.mat(y_train).T
        
        if self.intercept:
            ones = np.ones((self.X.shape[0], 1))
            self.X = np.hstack((ones, self.X))

        
        self.coef = np.random.uniform(low=-1, high=1, size=(self.X.shape[1], 1))
        self.gradient_descent()
        
    def gradient(self):
        """
            Helper function to calculate the gradient respect to coefficient.
            
            TODO: 5. Think about the matrix format of the gradient of the loss function.
        """
        y_pred = np.dot(self.X,self.coef)
        error = self.y-y_pred
        self.grad_coef = -2*np.dot(self.X.T, error)/self.X.shape[0]

        
    def gradient_descent(self):
        
        """
            Training function
            
            TODO: 6. Calculate the loss with current coefficients.
                  7. Update the temp_coef with learning rate and gradient.
                  8. Calculate the loss with temp_coef.
                  9. Implement the self adeptive learning rate. 
                      a. If current error is less than previous error, increase learning rate by a factor 1.3. 
                         And update coef, with temp_coef.
                      b. If previous error is less than current error, decrease learning rate by a factor of 0.9.
                         Don't update coef.
                  10. Add the loss to loss list we create.
        """
        
        self.loss = []
        learning_rate = self.alpha
        
        for i in range(self.num_iter):

                
                
            self.gradient()

            
            previous_y_hat = np.dot(self.X,self.coef)
            
            
            temp_coef = self.coef - learning_rate*self.grad_coef
            
            
            ones = np.ones((1,self.X.shape[0]))
            # Matrix with 1's (1 x n), help with calculate the sum of a mattrix. hint: Think about dot product.
            
            pre_error = np.dot(ones,np.square(previous_y_hat-self.y))
            
            current_error = np.dot(ones,np.square(np.dot(self.X,temp_coef) - self.y))
#             pre_error = np.sum(np.square(previous_y_hat-self.y))
#             current_error = np.sum(np.square(np.dot(self.X,temp_coef) - self.y))

            
            ### This is the early stop, don't modify fllowing three lines.
            if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                self.coef = temp_coef
                return self
            
            if current_error <= pre_error:
                learning_rate*=1.3
                self.coef = temp_coef
            else:
                learning_rate*=0.9
                
            self.loss.append(current_error)
            
#             print(self.loss)
            
            if i % 10000 == 0:
                print('Iteration: ' +  str(i))
                print('Coef: '+ str(self.coef))
                print('Loss: ' + str(current_error))            
        return self
    
    def ind_predict(self, x: list):
        """
            Predict the value based on its feature vector x.

            Parameter:
            x: Matrix, array or list. Input feature point.
            
            Return:
                result: prediction of given data point
        """
        
        """
            TODO: 11. Implement the prediction function
        """
        
        x = np.array(x)
  
        result = np.dot(x,self.coef)
        
        return result[0][0,0]
    
    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            
            Parameter:
            X: Matrix, array or list. Input feature point.
            
            Return:
                ret: prediction of given data matrix
        """
        
        """
            TODO: 12. Make sure add the 1's column like we did to add intercept.
                  13. Revise the following for-loop to call ind_predict to get predictions.

        """
        
        ret = []
        X = np.mat(X)
        if self.intercept:
            ones = np.ones((self.X.shape[0], 1))
            X = np.hstack((ones, X))
        for x in X:
            ret.append(self.ind_predict(x))
        return ret
        
        

In [136]:
def min_max_normaliz(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [147]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() +  20
m,n = X.shape


#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [150]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

Iteration: 0
Coef: [[ 0.51371659]
 [-0.21934786]]
Loss: [[2.67462249e+22]]
Iteration: 10000
Coef: [[ 0.64218478]
 [30.02914876]]
Loss: [[18820.91586191]]
Iteration: 20000
Coef: [[ 0.72602831]
 [30.02889453]]
Loss: [[18658.02966652]]
Iteration: 30000
Coef: [[ 0.80954095]
 [30.02876746]]
Loss: [[18496.70212898]]
Iteration: 40000
Coef: [[ 0.89266351]
 [30.02877591]]
Loss: [[18337.42438589]]
Iteration: 50000
Coef: [[ 0.97542277]
 [30.02851392]]
Loss: [[18178.61857033]]
Iteration: 60000
Coef: [[ 1.05782663]
 [30.02853208]]
Loss: [[18021.50915237]]
Iteration: 70000
Coef: [[ 1.13989781]
 [30.02839613]]
Loss: [[17865.3786168]]
Iteration: 80000
Coef: [[ 1.22158706]
 [30.02814995]]
Loss: [[17710.9277499]]
Iteration: 90000
Coef: [[ 1.30292576]
 [30.02815785]]
Loss: [[17558.40949111]]
Iteration: 100000
Coef: [[ 1.38390847]
 [30.02789958]]
Loss: [[17406.39400404]]
Iteration: 110000
Coef: [[ 1.46454385]
 [30.02792209]]
Loss: [[17255.96209088]]
Iteration: 120000
Coef: [[ 1.54482293]
 [30.02766375]]
L

####  As the number of iteration increase, you should notice the coeficient converges to [20, 30]. 
#### It maybe very slow update. Feel free to stop.

In [151]:
clf.coef

matrix([[19.74681248],
        [30.00037952]])

In [140]:
np.array(clf.predict(X))

array([   30.61958353,   180.76572308,   330.91186262,   481.05800217,
         631.20414172,   781.35028126,   931.49642081,  1081.64256036,
        1231.78869991,  1381.93483945,  1532.080979  ,  1682.22711855,
        1832.3732581 ,  1982.51939764,  2132.66553719,  2282.81167674,
        2432.95781629,  2583.10395583,  2733.25009538,  2883.39623493,
        3033.54237447,  3183.68851402,  3333.83465357,  3483.98079312,
        3634.12693266,  3784.27307221,  3934.41921176,  4084.56535131,
        4234.71149085,  4384.8576304 ,  4535.00376995,  4685.1499095 ,
        4835.29604904,  4985.44218859,  5135.58832814,  5285.73446769,
        5435.88060723,  5586.02674678,  5736.17288633,  5886.31902587,
        6036.46516542,  6186.61130497,  6336.75744452,  6486.90358406,
        6637.04972361,  6787.19586316,  6937.34200271,  7087.48814225,
        7237.6342818 ,  7387.78042135,  7537.9265609 ,  7688.07270044,
        7838.21883999,  7988.36497954,  8138.51111908,  8288.65725863,
      

#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [141]:
from sklearn.linear_model import LinearRegression

In [142]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density','alcohol']]
y = wine.quality

In [143]:
lr = LinearRegression()
lr.fit(X,y)
## Squared Error with sklearn.
sum((lr.predict(X) - y)**2)

800.6676988774323

#### You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [144]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X,y)

Iteration: 0
Coef: [[ 0.07749719]
 [ 0.74933981]
 [-0.7378636 ]]
Loss: [[1.2480774e+10]]
Iteration: 10000
Coef: [[0.45229701]
 [1.12999237]
 [0.38916866]]
Loss: [[806.90160285]]
Iteration: 20000
Coef: [[0.54372736]
 [1.22661051]
 [0.37111283]]
Loss: [[805.66799088]]
Iteration: 30000
Coef: [[0.57427437]
 [1.26196726]
 [0.36494882]]
Loss: [[805.51653202]]
Iteration: 40000
Coef: [[0.583471  ]
 [1.275842  ]
 [0.36274221]]
Loss: [[805.49729297]]
Iteration: 50000
Coef: [[0.58519043]
 [1.2821918 ]
 [0.36196671]]
Loss: [[805.49436963]]
Iteration: 60000
Coef: [[0.58429326]
 [1.28590873]
 [0.36170687]]
Loss: [[805.49342823]]
Iteration: 70000
Coef: [[0.58247784]
 [1.28870141]
 [0.36160962]]
Loss: [[805.49273172]]
Iteration: 80000
Coef: [[0.58034206]
 [1.29117193]
 [0.36158435]]
Loss: [[805.492068]]
Iteration: 90000
Coef: [[0.57809279]
 [1.29352886]
 [0.36157367]]
Loss: [[805.49140513]]
Iteration: 100000
Coef: [[0.57580442]
 [1.29584508]
 [0.36156681]]
Loss: [[805.49074739]]
Iteration: 110000
Coef

In [145]:
sum((clf.predict(X) - y)**2)

805.1777643607135

In [160]:
from sklearn.model_selection import train_test_split

In [152]:
df = pd.read_csv('diabetes_binary_health_indicators_BRFSS2015.csv')
df_clean = df.drop(columns = ['Fruits','AnyHealthcare','Sex','NoDocbcCost'])

In [169]:
X = df.drop(columns = ['Diabetes_binary'])
y = df['Diabetes_binary']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = Linear_Regression(alpha = 1, num_iter = 10000)
clf.fit(X,y)

Iteration: 0
Coef: [[-0.00174521]
 [ 0.66820189]
 [ 0.57301979]
 [ 0.49410739]
 [-0.10648133]
 [-0.9809506 ]
 [-0.46149868]
 [ 0.75356273]
 [-0.47966033]
 [-0.19358685]
 [ 0.19591478]
 [ 0.27334932]
 [ 0.30804159]
 [ 0.76402898]
 [-0.6965768 ]
 [-0.55413269]
 [ 0.76125117]
 [ 0.96012789]
 [ 0.41394132]
 [-0.83270099]
 [-0.106821  ]
 [ 0.07703525]]
Loss: [[8.31036566e+13]]


In [170]:
sum((clf.predict(X) - y)**2)

57511.56139549863

In [166]:
y_test

219620    0.0
132821    0.0
151862    0.0
139717    0.0
239235    0.0
         ... 
58678     0.0
203840    1.0
206241    0.0
53188     0.0
53404     0.0
Name: Diabetes_binary, Length: 76104, dtype: float64

In [167]:
X_test

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
219620,0.0,0.0,1.0,21.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,3.0,7.0,0.0,0.0,7.0,4.0,2.0
132821,1.0,1.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,0.0,13.0,6.0,6.0
151862,0.0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,7.0
139717,0.0,0.0,1.0,27.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,1.0,2.0,4.0,7.0
239235,0.0,1.0,1.0,31.0,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,4.0,27.0,27.0,1.0,0.0,8.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58678,0.0,0.0,1.0,29.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,2.0,0.0,0.0,0.0,1.0,7.0,6.0,8.0
203840,1.0,0.0,1.0,43.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,1.0,9.0,6.0,7.0
206241,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,5.0,6.0,7.0
53188,0.0,0.0,1.0,33.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,2.0,2.0,2.0,0.0,0.0,12.0,6.0,7.0


0.0